In [ ]:
import joblib
import click
import json
import time
import os
import itertools
import collections.abc
from shutil import copy,SameFileError
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pennylane as qml
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from qiskit_ibm_provider import IBMProvider

from quantum.Quantum import QuantumRegressor
from quantum.Evaluate import evaluate
from settings import ANSATZ_LIST, ENCODER_LIST
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import InconsistentVersionWarning

In [ ]:
def circuit(features, parameters):
    #  builds the circuit with the given encoder and variational circuits.
    #  encoder and variational circuits must have only two required parameters, params/feats and wires
    for i in range(re_upload_depth):
        params = parameters.reshape(re_upload_depth,-1)[i]
        
        encoder(features, wires=range(num_qubits))
        variational(params, wires=range(num_qubits))

    return qml.expval(qml.PauliZ(0))


In [ ]:
num_qubits=16
re_upload_depth=1
LAYERS=1
# device='qiskit.aer'
# device_backend = FakeCairoV2()
# backend = AerSimulator.from_backend(device_backend)
# device = qml.device(device, wires=num_qubits, backend=backend, shots=shots)


device='qulacs.simulator'
device = qml.device(device, wires=num_qubits, expansion_strategy="device")


encoder=ENCODER_LIST['IQP']
# encoder=ENCODER_LIST['M-M-CZ']
# encoder=ENCODER_LIST['A2-A2-CNOT']

# variational=ANSATZ_LIST['HWE-CZ']
variational=ANSATZ_LIST['Full-CRX']
variational.set_wires(range(num_qubits))
variational.layers = LAYERS


num_params = variational.num_params * re_upload_depth
generator = np.random.default_rng(12958234)
initial_parameters = generator.uniform(-np.pi, np.pi, num_params)

qnode = qml.QNode(circuit, device)

circuit_specs=qml.specs(qnode)(generator.uniform(-1, 1, num_qubits),initial_parameters)

print(circuit_specs['resources'])

qml.draw_mpl(qnode,level=1)(generator.uniform(-1, 1, num_qubits),initial_parameters)
# plt.savefig('trash.png')
plt.show()

In [ ]:

# qml.draw_mpl(qml.QNode(encoder,device),level=1)(generator.uniform(-1, 1, num_qubits),initial_parameters)
draw_encoder = qml.draw_mpl(qml.QNode(encoder,device),level=3)
draw_encoder(np.random.rand(16), range(16))
# plt.savefig('IQP_deep.png',bbox_inches='tight')

In [ ]:
qml.specs(qml.QNode(encoder,device,expansion_strategy='device'))(np.random.rand(num_qubits), range(num_qubits))['resources']

In [ ]:
qml.specs(qml.QNode(variational,device,expansion_strategy='device'))(np.random.rand(variational.num_params))['resources']

In [ ]:
variational.num_params

In [ ]:
num_params / 16

In [ ]:
2**num_qubits

In [ ]:
initial_parameters.shape

In [ ]:
# Use the 'qml.gradients.param_shift' to compute the gradient tape and extract the trainable params
gradient_fn = qml.gradients.param_shift(circuit)
gradient = gradient_fn(generator.uniform(-1, 1, num_qubits),initial_parameters)

# Count the number of trainable parameters
trainable_params = gradient.tape.trainable_params

# Print the number of trainable parameters
print(f"Number of trainable parameters: {len(trainable_params)}")

In [ ]:
check={}
bad=[]
for ke,ve in ENCODER_LIST.items():
    check[ke]={}
    for ka,va in ANSATZ_LIST.items():
        check[ke][ka]={}
        for i in [1,2]:
            LAYERS=i
            encoder=ve
            
            # variational=ANSATZ_LIST['Full-Pauli-CRZ']
            variational=va
            variational.layers = LAYERS
            variational.set_wires(range(num_qubits))
            
            
            # print(ka,variational.layers)
            num_params = variational.num_params * re_upload_depth
            generator = np.random.default_rng(12958234)
            initial_parameters = generator.uniform(-np.pi, np.pi, num_params)
            
            qnode = qml.QNode(circuit, device)
            
            circuit_specs=qml.specs(qnode)(generator.uniform(-1, 1, num_qubits),initial_parameters)
            
            check[ke][ka][i]=circuit_specs['resources'].depth
            
        if check[ke][ka][2]<check[ke][ka][1]:
            # print(ka)
            bad.append(ka)

In [ ]:
check

In [ ]:
for b in set(bad):
    print(ANSATZ_LIST[b])

In [ ]:
set(bad)